In [35]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [36]:
def calc_sma_and_bollinger_bands(df, sma_length):
    #Create a sma column for the df, setting all the values to NaN
    df["SMA"+str(sma_length)] = np.nan
    #Create an upper and lower bolinger band columns for the df, setting all the values to NaN
    df["Upper_Bollinger_Band"] = np.nan
    df["Lower_Bollinger_Band"] = np.nan

    #Create variables for the sum and sma
    sma_sum = 0
    sma = 0
    std_dev_sum = 0

    #Loop through the df
    for i in range(0, len(df.index)):
        #If i is less then 20 just sum the value and let the sma stay NaN
        if i < sma_length:
            sma_sum += df.iloc[i]["Close_VET"]

        #If i is greater than 20 add the next value and subtract the value from 20 places before
        elif i > sma_length:
            #Add the next day
            sma_sum += df.iloc[i]["Close_VET"]

            #Subtract the day 20 days before
            sma_sum -= df.iloc[i-sma_length]["Close_VET"]

            #Get the sma and set the sma column
            sma = sma_sum / float(sma_length)
            df["SMA"+str(sma_length)].iat[i] = sma
            
            #Reset standard deviation variables
            std_dev_sum = 0
            
            #Calculate std deviation
            for j in range(20):
                #Find average distance from moving average squared
                std_dev_sum += (df["Close_VET"][i-j] - sma)**2
            
            #Divide sum by num elements in the sum and take the square root
            std_dev = (std_dev_sum / float(sma_length))**0.5
            
            
            #Set the upper and lower bollinger bands 2 std deviations from the average
            df["Upper_Bollinger_Band"].iat[i] = sma + 2 * std_dev
            df["Lower_Bollinger_Band"].iat[i] = sma - 2 * std_dev

In [37]:
def generate_structured_data(hist, num_hist_days):
    #Create empty list of col names
    column_names = []
    #Create columns beforehand to use iat later
    for j in range(num_hist_days):
            for key in hist.keys():    
                column_names.append("Target-" + str(num_hist_days-j) + "_" + str(key))


    # Create empty df to push data into
    full_data = pd.DataFrame(columns=column_names, index=range(hist.shape[0]-num_hist_days))        

    # Include target columns
    full_data["Regression_Target"] = np.nan
    full_data["Classification_Target"] = np.nan
    
    #Control the row number of full_data
    for i in range(hist.shape[0]-num_hist_days):
        #Variables to hold the row and col of the original df indicies
        hist_row = i
        hist_col = 0
        #Control the column number of full_data
        for j in range(full_data.shape[1]):
            #For debugging if needed
            #print("full_data:", i, j, "hist:", hist_row, hist_col)

            #Place the original data into the full_data df
            full_data.iat[i, j] = hist.iat[hist_row, hist_col]

            #Increment the col where the data is pulled from
            hist_col += 1

            #When the col indexer for the original df reaches the end, increment the row and reset the col to 0
            if(hist_col == hist.shape[1]):
                hist_row += 1
                hist_col = 0

        # Create target columns
        full_data.loc[:,"Regression_Target"].iat[i] = hist.iloc[i+num_hist_days]["Close_VET"]

        #If the close > open, positive candle => 1 for classification
        if(hist.iloc[i+num_hist_days]["Close_VET"] > hist.iloc[i+num_hist_days]["Open_VET"]):
            full_data.loc[:,"Classification_Target"].iat[i] = 1
        else:
            full_data.loc[:,"Classification_Target"].iat[i] = 0
            
    return full_data

In [38]:
def generate_additional_features(data, num_hist_days):
    #Change each of the columns to be in terms of the relative price of the first day
    for i in range(num_hist_days, 0, -1):
        data["Target-"+str(i)+"_High_VET"] = data["Target-"+str(i)+"_High_VET"].div(data["Target-"+str(num_hist_days)+"_Open_VET"])
        data["Target-"+str(i)+"_Low_VET"] = data["Target-"+str(i)+"_Low_VET"].div(data["Target-"+str(num_hist_days)+"_Open_VET"])
        data["Target-"+str(i)+"_Close_VET"] = data["Target-"+str(i)+"_Close_VET"].div(data["Target-"+str(num_hist_days)+"_Open_VET"])
        data["Target-"+str(i)+"_SMA"+str(num_hist_days)] = data["Target-"+str(i)+"_SMA"+str(num_hist_days)].div(data["Target-"+str(num_hist_days)+"_Open_VET"])
        data["Target-"+str(i)+"_Upper_Bollinger_Band"] = data["Target-"+str(i)+"_Upper_Bollinger_Band"].div(data["Target-"+str(num_hist_days)+"_Open_VET"])
        data["Target-"+str(i)+"_Lower_Bollinger_Band"] = data["Target-"+str(i)+"_Lower_Bollinger_Band"].div(data["Target-"+str(num_hist_days)+"_Open_VET"])


    #Create a column to keep the original open price, so that we can get back to the original values if we need to
    data["Target-"+str(num_hist_days)+"_Original_Open_VET"] = data["Target-"+str(num_hist_days)+"_Open_VET"]

    #Create columns to hold the original regression target and the regression target relative to the price of the first day
    data["Original_Regression_Target"] = data["Regression_Target"]
    data["Regression_Target"] = data["Regression_Target"].div(data["Target-"+str(num_hist_days)+"_Open_VET"])

    #Always change the open price last as it will effect how the others are divided
    data["Target-"+str(num_hist_days)+"_Open_VET"] = data["Target-"+str(num_hist_days)+"_Open_VET"].div(data["Target-"+str(num_hist_days)+"_Open_VET"])
    
    #Calculate the relative difference between the upper/lower bollinger bands/ SMA vs the open/high/low/close of each day
    for i in range(num_hist_days, 0, -1):
        data["Target-"+str(i)+"_Open_Difference_From_Upper_Bollinger_Band"] = data["Target-"+str(i)+"_Upper_Bollinger_Band"].sub(data["Target-"+str(i)+"_Open_VET"])
        data["Target-"+str(i)+"_High_Difference_From_Upper_Bollinger_Band"] = data["Target-"+str(i)+"_Upper_Bollinger_Band"].sub(data["Target-"+str(i)+"_High_VET"])
        data["Target-"+str(i)+"_Low_Difference_From_Upper_Bollinger_Band"] = data["Target-"+str(i)+"_Upper_Bollinger_Band"].sub(data["Target-"+str(i)+"_Low_VET"])
        data["Target-"+str(i)+"_Close_Difference_From_Upper_Bollinger_Band"] = data["Target-"+str(i)+"_Upper_Bollinger_Band"].sub(data["Target-"+str(i)+"_Close_VET"])

        data["Target-"+str(i)+"_Open_Difference_From_Lower_Bollinger_Band"] = data["Target-"+str(i)+"_Lower_Bollinger_Band"].sub(data["Target-"+str(i)+"_Open_VET"])
        data["Target-"+str(i)+"_High_Difference_From_Lower_Bollinger_Band"] = data["Target-"+str(i)+"_Lower_Bollinger_Band"].sub(data["Target-"+str(i)+"_High_VET"])
        data["Target-"+str(i)+"_Low_Difference_From_Lower_Bollinger_Band"] = data["Target-"+str(i)+"_Lower_Bollinger_Band"].sub(data["Target-"+str(i)+"_Low_VET"])
        data["Target-"+str(i)+"_Close_Difference_From_Lower_Bollinger_Band"] = data["Target-"+str(i)+"_Lower_Bollinger_Band"].sub(data["Target-"+str(i)+"_Close_VET"])

        data["Target-"+str(i)+"_Open_Difference_From_SMA"+str(num_hist_days)] = data["Target-"+str(i)+"_SMA"+str(num_hist_days)].sub(data["Target-"+str(i)+"_Open_VET"])
        data["Target-"+str(i)+"_High_Difference_From_SMA"+str(num_hist_days)] = data["Target-"+str(i)+"_SMA"+str(num_hist_days)].sub(data["Target-"+str(i)+"_High_VET"])
        data["Target-"+str(i)+"_Low_Difference_From_SMA"+str(num_hist_days)] = data["Target-"+str(i)+"_SMA"+str(num_hist_days)].sub(data["Target-"+str(i)+"_Low_VET"])
        data["Target-"+str(i)+"_Close_Difference_From_SMA"+str(num_hist_days)] = data["Target-"+str(i)+"_SMA"+str(num_hist_days)].sub(data["Target-"+str(i)+"_Close_VET"])

        
    # Return the dataframe
    return data

In [39]:
#Get the data
vet = yf.Ticker("VET-USD")
btc = yf.Ticker("BTC-USD")

#Get max daily data
hist_vet = yf.download("VET-USD", start="2018-08-03", end="2021-11-12")
hist_btc = yf.download("BTC-USD", start="2018-08-03", end="2021-11-12")

#Drop unecessary cols
hist_vet.drop(["Adj Close"], axis=1, inplace=True)
hist_btc.drop(["Adj Close"], axis=1, inplace=True)

#Combine the two dfs into 1
hist = hist_vet.join(hist_btc, lsuffix="_VET", rsuffix="_BTC")
print(hist_vet.head(1))
print(hist_btc.head(1))
print(hist.head(1))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                Open     High       Low     Close    Volume
Date                                                       
2018-08-03  0.015041  0.01573  0.014938  0.015307  46250600
                   Open         High          Low        Close      Volume
Date                                                                      
2018-08-03  7562.140137  7562.140137  7328.649902  7434.390137  4627150000
            Open_VET  High_VET   Low_VET  Close_VET  Volume_VET     Open_BTC  \
Date                                                                           
2018-08-03  0.015041   0.01573  0.014938   0.015307    46250600  7562.140137   

               High_BTC      Low_BTC    Close_BTC  Volume_BTC  
Date                                                           
2018-08-03  7562.140137  7328.649902  7434.390137  4627150000  


In [40]:
#Define sma size
sma_length = 20

#Get Bollinger Bands and SMA
calc_sma_and_bollinger_bands(hist, sma_length)

#Drop the front of the df with the NaN vals from calculating the sma
hist = hist.iloc[sma_length+1:]

In [43]:
#Define the number of historical days to use as features
num_hist_days = 20

#Create the structured df
data_for_csv = generate_structured_data(hist, num_hist_days)

data_for_csv.head()

,Target-20_Open_VET,Target-20_High_VET,Target-20_Low_VET,Target-20_Close_VET,Target-20_Volume_VET,Target-20_Open_BTC,Target-20_High_BTC,Target-20_Low_BTC,Target-20_Close_BTC,Target-20_Volume_BTC,...,Target-1_Open_BTC,Target-1_High_BTC,Target-1_Low_BTC,Target-1_Close_BTC,Target-1_Volume_BTC,Target-1_SMA20,Target-1_Upper_Bollinger_Band,Target-1_Lower_Bollinger_Band,Regression_Target,Classification_Target
0,0.014743,0.014743,0.013981,0.014535,19980000,6551.52002,6719.959961,6498.640137,6719.959961,4097820000,...,6317.009766,6363.870117,6265.089844,6351.799805,4064230000,0.016273,0.019782,0.012764,0.014060,1.0
1,0.014552,0.015735,0.014244,0.015413,24453700,6719.950195,6789.629883,6700.959961,6763.189941,3312600000,...,6354.240234,6535.410156,6354.240234,6517.310059,4210910000,0.016249,0.019811,0.012688,0.013978,0.0
2,0.01539,0.015446,0.014513,0.015233,20503900,6754.640137,6774.75,6620.75,6707.259766,3295500000,...,6515.410156,6596.100098,6456.169922,6512.709961,4076220000,0.016178,0.019857,0.012499,0.014275,1.0
3,0.015222,0.017856,0.01521,0.017824,53968100,6710.799805,6884.640137,6689.709961,6884.640137,4019000000,...,6509.399902,6561.720215,6493.549805,6543.200195,3216300000,0.01613,0.019879,0.01238,0.014170,0.0
4,0.017839,0.018756,0.017549,0.01798,45607800,6891.080078,7109.560059,6882.339844,7096.279785,4659940000,...,6536.680176,6544.330078,6460.100098,6517.180176,3273730000,0.015947,0.019699,0.012196,0.012725,0.0


In [45]:
#Create the additional features for the structured df
data_for_csv = generate_additional_features(data_for_csv, num_hist_days)

In [46]:
data_for_csv

,Target-20_Open_VET,Target-20_High_VET,Target-20_Low_VET,Target-20_Close_VET,Target-20_Volume_VET,Target-20_Open_BTC,Target-20_High_BTC,Target-20_Low_BTC,Target-20_Close_BTC,Target-20_Volume_BTC,...,Target-1_Low_Difference_From_Upper_Bollinger_Band,Target-1_Close_Difference_From_Upper_Bollinger_Band,Target-1_Open_Difference_From_Lower_Bollinger_Band,Target-1_High_Difference_From_Lower_Bollinger_Band,Target-1_Low_Difference_From_Lower_Bollinger_Band,Target-1_Close_Difference_From_Lower_Bollinger_Band,Target-1_Open_Difference_From_SMA20,Target-1_High_Difference_From_SMA20,Target-1_Low_Difference_From_SMA20,Target-1_Close_Difference_From_SMA20
0,1.0,1.0,0.948314,0.985892,19980000,6551.52002,6719.959961,6498.640137,6719.959961,4097820000,...,0.545844,0.403403,0.852612,-0.083217,0.069804,-0.072636,1.090632,0.154802,0.307824,0.165384
1,1.0,1.081295,0.978835,1.059167,24453700,6719.950195,6789.629883,6700.959961,6763.189941,3312600000,...,0.449117,0.395172,0.858104,-0.114661,-0.040307,-0.094252,1.102816,0.130051,0.204405,0.15046
2,1.0,1.003639,0.943015,0.989799,20503900,6754.640137,6774.75,6620.75,6707.259766,3295500000,...,0.410828,0.381979,0.798052,-0.120802,-0.067261,-0.096111,1.037097,0.118242,0.171784,0.142934
3,1.0,1.173039,0.999212,1.170937,53968100,6710.799805,6884.640137,6689.709961,6884.640137,4019000000,...,0.4024,0.368173,0.799363,-0.146408,-0.09024,-0.124466,1.045683,0.099911,0.15608,0.121853
4,1.0,1.051404,0.983744,1.007904,45607800,6891.080078,7109.560059,6882.339844,7096.279785,4659940000,...,0.329477,0.309913,0.66939,-0.11992,-0.091106,-0.11067,0.879682,0.090372,0.119185,0.099622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1139,1.0,1.006613,0.968115,0.979857,337277736,62043.164062,64434.535156,61622.933594,64261.992188,40471196346,...,0.051743,0.044548,0.800425,-0.406877,-0.341867,-0.349062,0.997229,-0.210072,-0.145062,-0.152257
1140,1.0,1.058901,0.992166,1.050912,413430313,64284.585938,66930.390625,63610.675781,65992.835938,40788955582,...,0.130039,-0.074634,0.82208,-0.537143,-0.330369,-0.535042,1.052284,-0.306939,-0.100165,-0.304838
1141,1.0,1.112216,0.999771,1.056979,1045478292,66002.234375,66600.546875,62117.410156,62210.171875,45908121370,...,0.004957,0.000686,0.757964,-0.600263,-0.462766,-0.467037,0.991826,-0.366402,-0.228904,-0.233175
1142,1.0,1.056178,0.97778,1.002807,688702052,62237.890625,63715.023438,60122.796875,60692.265625,38434082775,...,0.179852,0.081255,0.717836,-0.517668,-0.270397,-0.368994,0.94296,-0.292544,-0.045272,-0.143869


In [47]:
data_for_csv.to_csv("Structured_Data.csv", index=False)